Nos conectamos a la DB de Langfuse y hacemos queries sobre la tabla de trazas

Podemos descargarnos los datos totales de trazas pero solo con las columnas que nos importan. 

Para poder estimar cuales son los tags mas caros y demas informacion relevante necesitamos el script que habia hecho hace unos meses de estimación de costes. En este caso no lo voy a hacer para un tenant sino para un tag o serie de tags.

Preguntas:

* Cuantas trazas hay
* Cuantas trazas de los ultimos X dias que tienen la palabra prod o dev
* Cuanto nos gastamos en prod
* Cuanto nos gastamos en dev
* Cuanto nos gastamos en el resto de trazas
* Rankear las trazas no prod o dev

Cuan importante es este tema?
Queremos automatizarlo? En tal caso, como queremos hacerlo?
Queremos que le pase la tarea a Nekane?

----


Ventajas de hacer con script:

* Si tenemos claro lo que queremos ver, al tenerlo de forma programatica pues se ejecuta automaticamente y podemos "construir" sobre ello

Desventajas:
* Lleva tiempo de programar
* Va a cambiar lo que queremos ver?
* Vamos a dejar de usar langfuse?


-----

TODO: Hacer un notebook con algunas de estas preguntas resueltas, pedir un meeting con Adma y Juan mañana




In [16]:
import pandas as pd
from datetime import date, timedelta

from aily_ai_brain.common.secrets_manager import get_aws_secret_key
from sqlalchemy import create_engine, text
from sqlalchemy.orm import Session, sessionmaker
from aily_py_commons.aily_logging import aily_logging as logging
from aily_py_commons.io.env_vars import (
    INFRASTRUCTURE_PROD,
    AilySettings,
)

## Connect to DB

In [17]:
def create_db_session_standard(tenant_name: str, environment: str):
    """
    Retrieves the database info for the given tenant using AWS credentials,
    then, creates a new database session for it.
    """
    secret_name = f"aily/{tenant_name}/{environment}/rds-langfuse/dbo"
    creds = {k: get_aws_secret_key(secret_name, k) for k in ["username", "password", "host", "port", "dbName"]}

    dialect_driver = "postgresql+psycopg2://"
    engine_url = (
        f"{dialect_driver}{creds['username']}:{creds['password']}@{creds['host']}:{creds['port']}/{creds['dbName']}"
    )
    sql_engine = create_engine(engine_url)
    sql_session = sessionmaker(bind=sql_engine)
    return sql_session

In [18]:
AilySettings(INFRASTRUCTURE_PROD)

2024-07-17 10:36:01 CEST+0200 - WARNING - aily-logging: You are using a dictionary to configure AilySettings. This is not recommended.Please use .env file to set your env variables in your environment.


AilySettings(AILY_PREFIX='aily', AILY_ENV='prod', AILY_TENANT='infrastructure', AWS_REGION='eu-central-1', AWS_PROFILE='aws-infrastructure')

In [19]:
tenant_name = "infrastructure"
environment = "prod"
tenant_session = create_db_session_standard(tenant_name, environment)

/var/folders/47/gybx85kd6f147h3rp8wgdz9w0000gn/T/ipykernel_71203/3766700193.py:7: DeprecationWarning: Please use `aily_data_aws.aws.secrets.get_secret()`
  creds = {k: get_aws_secret_key(secret_name, k) for k in ["username", "password", "host", "port", "dbName"]}


## Download data

In [20]:
def get_traces(session: Session, columns: list[str] = [], ids: list[int] = [], from_date: date = None) -> pd.DataFrame:
    """
    Return traces without the "input", "output",
    and "project_id" columns by default because they will be overwritten
    """
    default_columns = [
        "id",
        "timestamp",
        "tags",
    ]

    if not columns:
        selected_columns = default_columns
    else:
        selected_columns = columns

    selected_columns_text = ", ".join(f'"{col}"' for col in selected_columns)

    conditions = []
    params = {}

    if ids:
        conditions.append("id = ANY (ARRAY[:ids])")
        params["ids"] = ids

    if from_date:
        conditions.append("created_at >= :from_date")
        params["from_date"] = from_date

    if conditions:
        query_where = " WHERE " + " AND ".join(conditions)
    else:
        query_where = ""

    query = text(
        f"""
        SELECT {selected_columns_text}
        FROM public.traces
        {query_where}
        """
    )

    result = session.execute(query, params=params)

    df = pd.DataFrame(result.fetchall(), columns=result.keys())
    return df

def get_generation_observations(
    session: Session, columns: list[str] = [], ids: list[int] = [], from_date: date = None
) -> pd.DataFrame:
    """
    Return observations of type generation filtered by columns and ids.
    """
    default_columns = [
        "id",
        "type",
        "trace_id",
        "completion_tokens",
        "prompt_tokens",
        "total_tokens",
        "version",
        "created_at",
        "unit",
        "internal_model",
    ]

    if not columns:
        selected_columns = default_columns
    else:
        selected_columns = columns

    selected_columns_text = ", ".join(f'"{col}"' for col in selected_columns)

    conditions = ["type = 'GENERATION'"]
    params = {}

    if ids:
        conditions.append("id = ANY (ARRAY[:ids])")
        params["ids"] = ids

    if from_date:
        conditions.append("created_at >= :from_date")
        params["from_date"] = from_date

    query_where = " WHERE " + " AND ".join(conditions)

    query = text(
        f"""
        SELECT {selected_columns_text}
        FROM public.observations
        {query_where}
        """
    )

    result = session.execute(query, params=params)

    df = pd.DataFrame(result.fetchall(), columns=result.keys())
    return df

def get_models(session: Session, columns: list[str] = []) -> pd.DataFrame:
    """
    Return models and their costs.
    """
    if not columns:
        # Select all columns if columns list is empty
        query = text(
            """SELECT *
            FROM public.models
            """
        )
    else:
        # Select specified columns
        selected_columns_text = ", ".join(f'"{col}"' for col in columns)
        query = text(
            f"""SELECT {selected_columns_text}
            FROM public.models
            """
        )

    result = session.execute(query)
    df = pd.DataFrame(result.fetchall(), columns=result.keys())
    return df

In [21]:
today = date.today()
from_date = today - timedelta(days=7)

with tenant_session() as session:
    print("Downloading models:")
    df_models = get_models(session)
    
    print("Downloading traces:")
    df_traces = get_traces(session=session, from_date=from_date)
    
    print("Downloading observations:")
    df_generation_observations = get_generation_observations(session, from_date=from_date)
    

In [22]:
df_generation_observations

,id,type,trace_id,completion_tokens,prompt_tokens,total_tokens,version,created_at,unit,internal_model
0,coreproduct-938c6287-0f2a-4126-8077-5e8b9f15fede,GENERATION,coreproduct-fec1088a-28d1-4bfd-b596-df469bc5d039,134,219,353,None,2024-07-11 09:05:33.758,TOKENS,gpt-4o
1,coreproduct-f7662114-4065-49e7-b4e1-a80ff88aec9d,GENERATION,coreproduct-6fc7a43d-8f7c-4483-8f02-5e046125309a,69,498,567,None,2024-07-15 12:23:24.708,TOKENS,gpt-4o
2,coreproduct-a91ac806-6a77-46de-ae3b-cda106fbdf13,GENERATION,coreproduct-4d964d3a-b09a-40b0-85b6-dac2f9b4e42d,64,929,993,None,2024-07-15 12:23:24.726,TOKENS,gpt-4o
3,coreproduct-7ec8a286-48e8-4161-9d21-7db312e5d248,GENERATION,coreproduct-6fc7a43d-8f7c-4483-8f02-5e046125309a,69,495,564,None,2024-07-15 12:23:24.728,TOKENS,gpt-4o
4,coreproduct-6edcbaf3-685c-4ead-bd46-bfbf795605fd,GENERATION,coreproduct-4d964d3a-b09a-40b0-85b6-dac2f9b4e42d,71,900,971,None,2024-07-15 12:23:24.745,TOKENS,gpt-4o
...,...,...,...,...,...,...,...,...,...,...
61349,coreproduct-457e481a-3a84-4fa8-943f-80a1dcbf80e9,GENERATION,coreproduct-ff7b4fb2-5373-4638-a55d-23a0518f7938,0,4749,4749,None,2024-07-17 08:00:09.923,TOKENS,gpt-3.5-turbo
61350,coreproduct-7263753c-5a1c-4d6b-842e-43e2bb274f51,GENERATION,coreproduct-37126f2f-6438-4381-b4f4-28b99a271f4b,0,7291,7291,None,2024-07-17 08:00:10.047,TOKENS,gpt-3.5-turbo
61351,coreproduct-af45c321-8a8d-414c-984a-b7d6ab91ec71,GENERATION,coreproduct-67d2c9fa-a281-42b8-a1e6-c9b094ca6ba5,0,1719,1719,None,2024-07-17 08:00:10.578,TOKENS,gpt-3.5-turbo
61352,coreproduct-4eaa96cf-c373-4021-af8c-cdbd96deabcc,GENERATION,coreproduct-e16efc1b-c419-49e7-bc0d-a3ba90948ba5,0,6275,6275,None,2024-07-17 08:00:12.522,TOKENS,gpt-3.5-turbo


## Estimate cost of each observation

In order to estimate the cost of each observation, we need to multiply the `input_price` of the model by the total number of `prompt_tokens`and the `output_price` by the number of `completion_tokens`

In [23]:
def calculate_cost(row, df_models):
  model_info = df_models[df_models["model_name"] == row["internal_model"]]
  
  if model_info.empty:
    return 0

  observation_date = pd.to_datetime(row["created_at"])
  model_prices = model_info[["start_date", "input_price", "output_price"]]
  
  cost = 0
  
  if model_prices.empty:
    return 0
  
  elif model_prices.shape[0] == 1:
    token_prices = model_prices.iloc[0]
    input_price = token_prices["input_price"]
    output_price = token_prices["output_price"]
    cost = row["prompt_tokens"] * input_price + row["completion_tokens"] * output_price
    
  elif model_prices.shape[0] > 1:
    most_recent_prices = model_prices[model_prices["start_date"] <= observation_date].sort_values(by="start_date", ascending=False)
    # print(most_recent_prices)
    if most_recent_prices.empty:
      # TODO: We could also return a cost of 0
      raise Exception(f"The observation {row} is younger than any of the multiple associated prices")
    else:
      token_prices = most_recent_prices.sort_values(by="start_date", ascending=False).iloc[0]
      input_price = token_prices["input_price"]
      output_price = token_prices["output_price"]
      cost = row["prompt_tokens"] * input_price + row["completion_tokens"] * output_price
  
  return cost

In [24]:
df_generation_observations.loc[:, "cost"] = df_generation_observations.apply(calculate_cost, axis=1, args=(df_models,))

## Question answering

* Cuantas trazas hay
* Cuantas trazas de los ultimos X dias que tienen la palabra prod o dev
* Cuanto nos gastamos en prod
* Cuanto nos gastamos en dev
* Cuanto nos gastamos en el resto de trazas
* Rankear las trazas no prod o dev


### How many unique tags are? 

In [29]:
print(f"There are {df_traces['tags'].value_counts().shape[0]} unique tag combinations")

There are 122 unique tag combinations


In [30]:
# Step 1: Flatten the list values
all_values = [item for sublist in df_traces['tags'] for item in sublist]

# Step 2: Find unique values
unique_values = set(all_values)

# Step 3: Count unique values
unique_count = len(unique_values)

print(f"Number of unique tags: {unique_count}")

Number of unique tags: 104


###  How many traces with the word `prod` or `dev`

In [39]:
def how_many_contain_the_tag(df, tag_name):
    # Check each list for the presence of the number 2
    contains_it = df['tags'].apply(lambda x: tag_name in x)
    
    # Sum the boolean values to get the count
    count_rows_with_it = contains_it.sum()
    
    print(f"Number of rows containing the tag '{tag_name}': {count_rows_with_it}/{df.shape[0]}")
    
    return count_rows_with_it
    
def how_many_do_not_contain_tags(df, tags_list):
    # Check each list for the absence of all tags in tags_list
    does_not_contain_any = df['tags'].apply(lambda x: all(tag not in x for tag in tags_list))
    
    # Sum the boolean values to get the count
    count_rows_without_tags = does_not_contain_any.sum()
    
    print(f"Number of rows not containing any of the tags {tags_list}: {count_rows_without_tags}/{df.shape[0]}")
    
    return count_rows_without_tags

def how_many_contain_all_tags(df, tags_list):
    # Check each list for the presence of all tags in tags_list
    does_not_contain_any = df['tags'].apply(lambda x: all(tag in x for tag in tags_list))
    
    # Sum the boolean values to get the count
    count_rows_with_tags = does_not_contain_any.sum()
    
    print(f"Number of rows containing all of the tags {tags_list}: {count_rows_with_tags}/{df.shape[0]}")
    
    return count_rows_with_tags
    
n_1 = how_many_contain_the_tag(df_traces, "prod")
n_2 = how_many_contain_the_tag(df_traces, "dev")
n_3 = how_many_do_not_contain_tags(df_traces, ["prod", "dev"])
n_4 = how_many_contain_all_tags(df_traces, ["prod", "dev"])

# We can see that the calculations are correct, there are 2 traces with both dev and prod tags
n_1 + n_2 + n_3

Number of rows containing the tag 'prod': 35362/41989
Number of rows containing the tag 'dev': 4471/41989
Number of rows not containing any of the tags ['prod', 'dev']: 2158/41989
Number of rows containing all of the tags ['prod', 'dev']: 2/41989


41991

### Rank unique trace combinations according to `cost`

In [40]:
df_generation_observations

,id,type,trace_id,completion_tokens,prompt_tokens,total_tokens,version,created_at,unit,internal_model,cost
0,coreproduct-938c6287-0f2a-4126-8077-5e8b9f15fede,GENERATION,coreproduct-fec1088a-28d1-4bfd-b596-df469bc5d039,134,219,353,None,2024-07-11 09:05:33.758,TOKENS,gpt-4o,0.003105000000000000000000000000
1,coreproduct-f7662114-4065-49e7-b4e1-a80ff88aec9d,GENERATION,coreproduct-6fc7a43d-8f7c-4483-8f02-5e046125309a,69,498,567,None,2024-07-15 12:23:24.708,TOKENS,gpt-4o,0.003525000000000000000000000000
2,coreproduct-a91ac806-6a77-46de-ae3b-cda106fbdf13,GENERATION,coreproduct-4d964d3a-b09a-40b0-85b6-dac2f9b4e42d,64,929,993,None,2024-07-15 12:23:24.726,TOKENS,gpt-4o,0.005605000000000000000000000000
3,coreproduct-7ec8a286-48e8-4161-9d21-7db312e5d248,GENERATION,coreproduct-6fc7a43d-8f7c-4483-8f02-5e046125309a,69,495,564,None,2024-07-15 12:23:24.728,TOKENS,gpt-4o,0.003510000000000000000000000000
4,coreproduct-6edcbaf3-685c-4ead-bd46-bfbf795605fd,GENERATION,coreproduct-4d964d3a-b09a-40b0-85b6-dac2f9b4e42d,71,900,971,None,2024-07-15 12:23:24.745,TOKENS,gpt-4o,0.005565000000000000000000000000
...,...,...,...,...,...,...,...,...,...,...,...
61349,coreproduct-457e481a-3a84-4fa8-943f-80a1dcbf80e9,GENERATION,coreproduct-ff7b4fb2-5373-4638-a55d-23a0518f7938,0,4749,4749,None,2024-07-17 08:00:09.923,TOKENS,gpt-3.5-turbo,0.002374500000000000000000000000
61350,coreproduct-7263753c-5a1c-4d6b-842e-43e2bb274f51,GENERATION,coreproduct-37126f2f-6438-4381-b4f4-28b99a271f4b,0,7291,7291,None,2024-07-17 08:00:10.047,TOKENS,gpt-3.5-turbo,0.003645500000000000000000000000
61351,coreproduct-af45c321-8a8d-414c-984a-b7d6ab91ec71,GENERATION,coreproduct-67d2c9fa-a281-42b8-a1e6-c9b094ca6ba5,0,1719,1719,None,2024-07-17 08:00:10.578,TOKENS,gpt-3.5-turbo,0.000859500000000000000000000000
61352,coreproduct-4eaa96cf-c373-4021-af8c-cdbd96deabcc,GENERATION,coreproduct-e16efc1b-c419-49e7-bc0d-a3ba90948ba5,0,6275,6275,None,2024-07-17 08:00:12.522,TOKENS,gpt-3.5-turbo,0.003137500000000000000000000000


### Rank tag combinations according to `cost`

In [46]:
merged_df = df_generation_observations.merge(df_traces, left_on='trace_id', right_on='id')

# Group by the 'tags' column and aggregate the costs
tag_combinations_cost = merged_df.groupby(merged_df['tags'].apply(tuple))['cost'].sum().reset_index()

# Rename columns for clarity
tag_combinations_cost.columns = ['tags', 'total_cost']

tag_combinations_cost.sort_values(["total_cost"], ascending=False)

,tags,total_cost
74,"(core, extract_content, prnewswire, prod, scan...",93.91665850000000000000000000
27,"(auto_insights, dev, genai, sql_agent_langgraph)",24.57265000000000000000000000
69,"(core, extract_content, globenewswire, prod, s...",22.26861000000000000000000000
29,"(auto_insights, dev, sql_agent_langgraph)",19.63242500000000000000000000
103,"(dev, post-processing, scanner)",14.34718500000000000000000000
...,...,...
7,"(aily-ai-brain, dev, genai, test)",0.001008000000000000000000000000
11,"(ailybot, dev, genai, gpt)",0.000990000000000000000000000000
36,"(brain_example_langfuse, genai, prod)",0.000243000000000000000000000000
34,"(brain_example_langfuse, dev)",0.000028000000000000000000000000


### Rank unique tags according to `cost`

In [47]:
# Expand the 'tags' column so each tag has its own row
expanded_df = merged_df.explode('tags')

# Group by the 'tags' column and aggregate the costs
unique_tags_cost = expanded_df.groupby('tags')['cost'].sum().reset_index()

# Rename columns for clarity
unique_tags_cost.columns = ['tag', 'total_cost']

unique_tags_cost.sort_values(["total_cost"], ascending=False)

,tag,total_cost
75,scanner,226.3979920000000000000000000
68,prod,216.7081727500000000000000000
25,core,212.0508070000000000000000000
33,extract_content,168.0417960000000000000000000
30,dev,144.6937685000000000000000000
...,...,...
93,teva_wishlist_test_PBF,0.003488000000000000000000000000
56,infrastructure,0.003130000000000000000000000000
6,aily_ai_brain,0.003130000000000000000000000000
20,brain_example_langfuse,0.002514000000000000000000000000
